<p align="center">
  <img src="https://i.ibb.co/5Y2YwpC/313132131.png" alt="UFPA" width=100/>
  <img src="https://i.ibb.co/wwXNYvM/abc.png" alt="ENGCOMP" width=100/>
</p>

<p align="center">
  <b>SERVIÇO PÚBLICO FEDERAL<br>
  MINISTERIO DA EDUCAÇÃO<br>
  UNIVERSIDADE FEDERAL DO PARÁ<br>
  CAMPUS TUCURUÍ<br>
  ENGENHARIA DA COMPUTAÇÃO</b>
</p>

---
> **Professor**: Cleison Daniel Silva<br>
> **Aluno**: Danilo de Sousa Lopes<br>
> **Data**: 09/02/2023<br>

#**Treinamento - Implementação da Otimização Bayesiana em Sistemas BCI-IM**

###**Introdução**
> Este treinamento tem como objetivo a demostração da implementação da Otimização Bayesiana em três possíveis cenários.

### **Environment**

In [ ]:
from google.colab import drive

# montagem do drive
drive.mount('/content/drive')

# download das dependências da biblioteca pyriemann e skopt
! pip install pyriemann scikit-optimize -q

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# imports
import numpy as np
from scipy.signal import butter, lfilter

# imports sklearn
from sklearn.metrics import accuracy_score
from sklearn import svm

# imports pyriemann
from pyriemann.classification import MDM
from pyriemann.estimation import Covariances

# otimization
from skopt import gp_minimize

### **Setup**

In [ ]:
# definindo o sujeito
suj = 1

#criando o link para downloado
download_train = "https://github.com/danilosl/classify_eeg/raw/main/Dataset%20Processed/Treino/A0"+str(suj)+"T.npy"
download_test = "https://github.com/danilosl/classify_eeg/raw/main/Dataset%20Processed/Teste/A0"+str(suj)+"E.npy"

# download do dataset
!wget -q $download_train
!wget -q $download_test

# frequência de amostragem
Fs = 250

# carregando dados de treinamento
eeg_train = np.load('/content/A0'+str(suj)+'T.npy')

# carregando dados de teste
eeg_test = np.load('/content/A0'+str(suj)+'E.npy')

# labels
label = np.concatenate((np.zeros(72),np.ones(72)))

In [ ]:
def cov_eeg(eeg_train, eeg_test):
  # estimando as matrizes de covariância
  cov_eeg_train = Covariances().transform(eeg_train)
  cov_eeg_test = Covariances().transform(eeg_test)

  # instanciando um classificador MDM
  mdm = MDM(metric=dict(mean='riemann', distance='riemann'))

  # ajustando o classificador
  mdm.fit(cov_eeg_train,label)

  # estimando as distâncias de Riemann
  X_train = mdm.transform(cov_eeg_train)
  Y_test = mdm.transform(cov_eeg_test)
  return X_train, Y_test

### **SVM - Classificação sem Otmização | Cenário 1**

In [ ]:
X_train, Y_test = cov_eeg(eeg_train, eeg_test)

cf_svm = svm.SVC(kernel='rbf', C=255e-3)
cf_svm.fit(X_train, label)
predict_svm = cf_svm.predict(Y_test)
acurracy_svm = accuracy_score(label, predict_svm)
print(f"Acurracy SVM: {acurracy_svm*100:.2f}%")

Acurracy SVM: 63.19%


### **SVM - Classificação com Otimização Bayesiana | Cenário 2**

In [ ]:
def treinar_modelo(params):
  kernel = params[0]
  C = params[1]
  gamma = params[2]
  print(params, "\n")

  cf_svm = svm.SVC(kernel=kernel, C=C, gamma=gamma)
  cf_svm.fit(X_train, label)
  predict_svm = cf_svm.predict(X_train)
  return -accuracy_score(label, predict_svm)

space = [('linear', 'rbf', 'poly'),
         (255e-3, 10, 100, 1000),
         (1e-7, 1e-4, 1e-2, 0.1)]

In [ ]:
resultados_svm1 = gp_minimize(treinar_modelo, space, random_state=1, verbose=1, n_calls=30, n_random_starts=10)
best_kernel, best_C, best_gamma = resultados_svm1.x[0], resultados_svm1.x[1], resultados_svm1.x[2]
cf_svm = svm.SVC(kernel=best_kernel, C=best_C, gamma=best_gamma)
cf_svm.fit(X_train, label)
predict_svm = cf_svm.predict(Y_test)
acurracy_svm = accuracy_score(label, predict_svm)
print("-------------------------------------------------\n")
print(f"Acurracy SVM Otimizado: {acurracy_svm*100:.2f}%")

Iteration No: 1 started. Evaluating function at random point.
['rbf', 1000.0, 1e-07] 

Iteration No: 1 ended. Evaluation done at random point.
Time taken: 0.0061
Function value obtained: -0.5208
Current minimum: -0.5208
Iteration No: 2 started. Evaluating function at random point.
['rbf', 10.0, 0.0001] 

Iteration No: 2 ended. Evaluation done at random point.
Time taken: 0.0075
Function value obtained: -0.5208
Current minimum: -0.5208
Iteration No: 3 started. Evaluating function at random point.
['poly', 100.0, 0.1] 

Iteration No: 3 ended. Evaluation done at random point.
Time taken: 0.0068
Function value obtained: -0.9028
Current minimum: -0.9028
Iteration No: 4 started. Evaluating function at random point.
['rbf', 10.0, 0.01] 

Iteration No: 4 ended. Evaluation done at random point.
Time taken: 0.0062
Function value obtained: -0.5833
Current minimum: -0.9028
Iteration No: 5 started. Evaluating function at random point.
['poly', 10.0, 0.01] 

Iteration No: 5 ended. Evaluation done at

### **SVM - Classificação com Otimização de Parâmetros Externos| Cenário 3**

Imagine que é necessário filtrar os sinais de EEG e existem alguns ajustes de parâmetros do filtro, como a ordem e faixa de frênquencia alta e baixa. Neste caso também é possível otimizar os parâmetros do filtro em cojunto com os parâmentos do classificador.


In [ ]:
def new_model(eeg_train, eeg_test, label, N, fl, fh, kernel, C, gamma):
  # Instanciado um filtro Butterworth passa faixa.
  b, a = butter(N,[fl,fh], btype='bandpass', analog=False, fs=Fs)

  #Filtrando todos os canais do dataset
  eeg_train_filter = lfilter(b,a, eeg_train)
  eeg_test_filter = lfilter(b,a, eeg_test)

  X_train, Y_train = cov_eeg(eeg_train_filter, eeg_test_filter)

  clf_svm = svm.SVC(kernel=kernel, C=C, gamma=gamma)
  clf_svm.fit(X_train, label)
  predict_svm = clf_svm.predict(X_train)

  return (predict_svm, cf_svm, Y_train)

In [ ]:
def treinar_modelo(params):
  kernel = params[0]
  C = params[1]
  gamma = params[2]
  N = params[3]
  fl = params[4]
  fh = params[5]
  print(params, "\n")

  predict_svm, clf_svm, Y_train = new_model(eeg_train, eeg_test, label, N, fl, fh, kernel, C, gamma)
  return -accuracy_score(label, predict_svm)

space = [('linear', 'rbf', 'poly'),
         (255e-3, 10, 100, 1000),
         (1e-7, 1e-4, 1e-2, 0.1),
         (4,8),
         (5,10),
         (30,35)]

In [ ]:
resultados_svm2 = gp_minimize(treinar_modelo, space, random_state=1, verbose=1, n_calls=30, n_random_starts=10)
best_kernel, best_C, best_gamma, best_N, best_fl, best_fh = resultados_svm2.x[0], resultados_svm2.x[1], resultados_svm2.x[2], resultados_svm2.x[3], resultados_svm2.x[4], resultados_svm2.x[5]

predict_svm, clf_svm, Y_test = new_model(eeg_train, eeg_test, label, best_N, best_fl, best_fh, best_kernel, best_C, best_gamma)

predict_svm_test = cf_svm.predict(Y_test)
acurracy_svm = accuracy_score(label, predict_svm_test)
print("-------------------------------------------------\n")
print(f"Acurracy SVM2 Otimizado: {acurracy_svm*100:.2f}%")

Iteration No: 1 started. Evaluating function at random point.
['rbf', 1000.0, 1e-07, 8, 6, 32] 

Iteration No: 1 ended. Evaluation done at random point.
Time taken: 0.5926
Function value obtained: -0.7083
Current minimum: -0.7083
Iteration No: 2 started. Evaluating function at random point.
['poly', 100.0, 0.1, 7, 7, 33] 

Iteration No: 2 ended. Evaluation done at random point.
Time taken: 0.6663
Function value obtained: -0.9583
Current minimum: -0.9583
Iteration No: 3 started. Evaluating function at random point.
['poly', 10.0, 0.01, 8, 7, 32] 

Iteration No: 3 ended. Evaluation done at random point.
Time taken: 0.7004
Function value obtained: -0.5625
Current minimum: -0.9583
Iteration No: 4 started. Evaluating function at random point.
['rbf', 100.0, 0.01, 7, 5, 33] 

Iteration No: 4 ended. Evaluation done at random point.
Time taken: 0.9572
Function value obtained: -0.8819
Current minimum: -0.9583
Iteration No: 5 started. Evaluating function at random point.
['rbf', 100.0, 0.01, 5, 